In [1]:
using JLD2
using FourierFlows
using FourierFlows: parsevalsum2
using Printf
using CairoMakie
using AbstractFFTs
include("RSWUtils.jl")
include("AnalysisUtils.jl")

mapfilter_snapshots (generic function with 1 method)

In [15]:
function compute_transfer_function(run_directory, grid)
    Nsnapshots = count_key_snapshots(run_directory, "rsw")
    
    f0, Cg2 = read_rsw_params(run_directory)
    ν, nν = read_rsw_dissipation(run_directory)
    D = -ν*grid.Krsq.^(nν)
    params = (; f=f0, Cg2, D)
    
   
    Eh_flux = zeros(Float64, grid.nkr, grid.nl)
    E3h_flux = zeros(Float64, grid.nkr, grid.nl)
    Egggh_flux = zeros(Float64, grid.nkr, grid.nl)
    Eggwh_flux = zeros(Float64, grid.nkr, grid.nl)
    Egwwh_flux = zeros(Float64, grid.nkr, grid.nl)
    Ewwwh_flux = zeros(Float64, grid.nkr, grid.nl)
    E3wwwh_flux = zeros(Float64, grid.nkr, grid.nl)

    snap_frames = (Nsnapshots-40):Nsnapshots
    
    times = zeros(length(snap_frames))
    array_idx = 1
    
    Φ₀, Φ₊, Φ₋ = compute_balanced_wave_bases(grid, (; f=f0, Cg2))
    Fr = 0
    for snap_idx = snap_frames
        if (snap_idx % 1 == 0)
            println(@sprintf("Computed frame %d/%d", snap_idx, Nsnapshots))
        end
        
        t, rsw_sol = load_key_snapshot(run_directory, "rsw", snap_idx)
        rsw_sol = convert.(Complex{Float64}, rsw_sol)
        times[array_idx] = t
        
        dealias!(rsw_sol, grid)
        
        uh = rsw_sol[:,:,1]
        vh = rsw_sol[:,:,2]
        ηh = rsw_sol[:,:,3]

        ((ugh, vgh, ηgh), (uwh, vwh, ηwh)) = wave_balanced_decomposition(uh, vh, ηh, grid, params)
        
        total_field = compute_derivatives(uh,  vh,  ηh,  grid, params)
        geo_field   = compute_derivatives(ugh, vgh, ηgh, grid, params)
        wave_field  = compute_derivatives(uwh, vwh, ηwh, grid, params)

        ug2_total = (FourierFlows.parsevalsum2(ugh, grid) + FourierFlows.parsevalsum2(vgh, grid))
        ug_rms = sqrt.(ug2_total/grid.Lx/grid.Ly)
        Fr = ug_rms / sqrt.(Cg2)
        qh  = total_field[end]
        qgh = geo_field[end]
        qwh = wave_field[end]

        total_quadratic_terms = compute_duvηdth(total_field, total_field, grid, params)
        gg_quadratic_terms = compute_duvηdth(geo_field,  geo_field,  grid, params)
        gw_quadratic_terms = compute_duvηdth(geo_field,  wave_field, grid, params) .+ compute_duvηdth(wave_field, geo_field, grid, params)
        ww_quadratic_terms = compute_duvηdth(wave_field, wave_field, grid, params)

        E = compute_flux_fields(uh, vh, ηh, qh, total_quadratic_terms, grid, params)
        Eggg = compute_flux_fields(ugh, vgh, ηgh, qgh, gg_quadratic_terms, grid, params)
    
        Eggw = compute_flux_fields(ugh, vgh, ηgh, qgh, gw_quadratic_terms, grid, params) 
        Ewgg = compute_flux_fields(uwh, vwh, ηwh, qwh, gg_quadratic_terms, grid, params)
    
        Egww = compute_flux_fields(ugh, vgh, ηgh, qgh, ww_quadratic_terms, grid, params)
        Ewwg = compute_flux_fields(uwh, vwh, ηwh, qwh, gw_quadratic_terms, grid, params)
    
        Ewww = compute_flux_fields(uwh, vwh, ηwh, qwh, ww_quadratic_terms, grid, params)

        E3 = compute_cubic_flux_fields(uh, vh, ηh, qh, total_quadratic_terms, grid, params)
        E3www = compute_cubic_flux_fields(uwh, vwh, ηwh, qwh, ww_quadratic_terms, grid, params)

        @. Eh_flux    += E
        @. E3h_flux   += E3
        @. Egggh_flux += Eggg
        @. Eggwh_flux += Eggw + Ewgg
        @. Egwwh_flux += Egww + Ewwg
        @. Ewwwh_flux += Ewww
        @. E3wwwh_flux += E3www
        array_idx += 1
    end
    return times, Eh_flux, E3h_flux, Egggh_flux, Eggwh_flux, Egwwh_flux, Ewwwh_flux, E3wwwh_flux, Fr
end

function compute_derivatives(uh, vh, ηh, grid, params)
    uxh = @. 1im * grid.kr * uh
    vxh = @. 1im * grid.kr * vh
    ηxh = @. 1im * grid.kr * ηh
    uyh = @. 1im * grid.l  * uh
    vyh = @. 1im * grid.l  * vh
    ηyh = @. 1im * grid.l  * ηh

    qh = @. 1im * grid.kr * vh - 1im * grid.l * uh - params.f * ηh
    
    u  = irfft(uh, grid.nx)
    v  = irfft(vh, grid.nx)
    η  = irfft(ηh, grid.nx)
    ux = irfft(uxh, grid.nx)
    vx = irfft(vxh, grid.nx)
    ηx = irfft(ηxh, grid.nx)
    uy = irfft(uyh, grid.nx)
    vy = irfft(vyh, grid.nx)
    ηy = irfft(ηyh, grid.nx)

    return (u, v, η, ux, vx, ηx, uy, vy, ηy, qh)
end
function compute_duvηdth(field_set_1, field_set_2, grid, params)
    u1, v1, η1, ux1, vx1, ηx1, uy1, vy1, ηy1 = field_set_1
    u2, v2, η2, ux2, vx2, ηx2, uy2, vy2, ηy2 = field_set_2
    ηh = rfft(η1)
    uh = rfft(u1)
    vh = rfft(v1)
    uxh = rfft(ux1)
    vyh = rfft(vy1)
    ηxh = rfft(ηx1)
    ηyh = rfft(ηy1)
    uuxh = rfft(u1 .* ux2)
    vuyh = rfft(v1 .* uy2)
    uvxh = rfft(u1 .* vx2)
    vvyh = rfft(v1 .* vy2)
    uηxh = rfft(u1 .* ηx2)
    vηyh = rfft(v1 .* ηy2)
    ηuh  = rfft(η1 .* u2)
    ηvh  = rfft(η1 .* v2)
    divηuh   = @. 1im * grid.kr * ηuh + 1im * grid.l * ηvh

    dudth = @. -uuxh - vuyh +  params.f * vh - params.Cg2 * ηxh
    dvdth = @. -uvxh - vvyh -  params.f * uh - params.Cg2 * ηyh
    dηdth = @. -divηuh - uxh - vyh

    return dudth, dvdth, dηdth
end

function compute_flux_fields(uh, vh, ηh, qh, quadratic_terms, grid, params)
    dudth, dvdth, dηdth = quadratic_terms

    one_plus_ηh = rfft(1 .+ irfft(ηh, grid.nx))
    Euh = @. conj(uh) * dudth
    Evh = @. conj(vh) * dvdth
    Eηh = @. conj(one_plus_ηh) * dηdth
    
    Eh = @. real((Euh + Evh) + params.Cg2 * Eηh)
    return Eh
end

function total_energy(uh, vh, ηh, grid, params)
    u = irfft(uh, grid.nx)
    v = irfft(vh, grid.nx)
    η = irfft(ηh, grid.nx)
    one_plus_η = 1 .+ η
    m1 = u .* sqrt.(one_plus_η)
    m2 = v .* sqrt.(one_plus_η)

    m1h = rfft(m1)
    m2h = rfft(m2)
    m3h = rfft(one_plus_η)

    return 0.5 * (abs2.(m1h) + abs2.(m2h) + params.Cg2*abs2.(m3h))
end

function compute_cubic_flux_fields(uh, vh, ηh, qh, quadratic_terms, grid, params)
    dudth, dvdth, dηdth = quadratic_terms
    u = irfft(uh, grid.nx)
    v = irfft(vh, grid.nx)
    η = irfft(ηh, grid.nx)
    dudt = irfft(dudth, grid.nx)
    dvdt = irfft(dvdth, grid.nx)
    dηdt = irfft(dηdth, grid.nx)
    one_plus_η  = 1 .+ η
    sqrt_one_plus_η = sqrt.(one_plus_η)
    dsqrt_one_plus_ηdt = 1 ./ (2*sqrt_one_plus_η).*dηdt

    m1 = u.*sqrt_one_plus_η
    m2 = v.*sqrt_one_plus_η

    m1h = rfft(m1)
    m2h = rfft(m2)
    one_plus_ηh = rfft(one_plus_η)

    dm1dt = dsqrt_one_plus_ηdt.*u + sqrt_one_plus_η.*dudt
    dm2dt = dsqrt_one_plus_ηdt.*v + sqrt_one_plus_η.*dvdt

    dm1dth = rfft(dm1dt)
    dm2dth = rfft(dm2dt)
    Em1h = @. conj(m1h) * dm1dth
    Em2h = @. conj(m2h) * dm2dth
    Eηh = @. conj(one_plus_ηh) * dηdth
    
    Eh = @. real((Em1h + Em2h) + params.Cg2 * Eηh)

    return Eh
end

compute_cubic_flux_fields (generic function with 1 method)

In [11]:
include("../../../utils/ExactRadialSpectrum.jl")
radii, weight_matrix = create_radialspectrum_weights(grid, 8);

In [18]:
run_directory = "/vast/nad9961/rsw/65276574/6"
nx, Lx = get_grid_size(run_directory, "rsw")
grid = TwoDGrid(; nx, Lx)
times, Eh_flux, E3h_flux, Egggh_flux, Eggwh_flux, Egwwh_flux, Ewwwh_flux, E3www_flux, Fr = compute_transfer_function(run_directory, grid)

Computed frame 630/670
Computed frame 631/670
Computed frame 632/670
Computed frame 633/670
Computed frame 634/670
Computed frame 635/670
Computed frame 636/670
Computed frame 637/670
Computed frame 638/670
Computed frame 639/670
Computed frame 640/670
Computed frame 641/670
Computed frame 642/670
Computed frame 643/670
Computed frame 644/670
Computed frame 645/670
Computed frame 646/670
Computed frame 647/670
Computed frame 648/670
Computed frame 649/670
Computed frame 650/670
Computed frame 651/670
Computed frame 652/670
Computed frame 653/670
Computed frame 654/670
Computed frame 655/670
Computed frame 656/670
Computed frame 657/670
Computed frame 658/670
Computed frame 659/670
Computed frame 660/670
Computed frame 661/670
Computed frame 662/670
Computed frame 663/670
Computed frame 664/670
Computed frame 665/670
Computed frame 666/670
Computed frame 667/670
Computed frame 668/670
Computed frame 669/670
Computed frame 670/670


([1632.3250732421875, 1632.593505859375, 1632.8619384765625, 1633.13037109375, 1633.3988037109375, 1633.667236328125, 1633.9356689453125, 1634.2041015625, 1634.4725341796875, 1634.740966796875  …  1640.646484375, 1640.9149169921875, 1641.183349609375, 1641.4517822265625, 1641.72021484375, 1641.9886474609375, 1642.257080078125, 1642.5255126953125, 1642.7939453125, 1643.0623779296875], [-4.136062247849321e6 2.2001721168706656e9 … -2.2886536978664777e8 -2.063159028118716e9; 2.4139639143584475e7 -1.5629867323226118e8 … 1.3236397947515723e8 9.00449071996434e7; … ; -1.182913144264888e-14 -1.8515763223882767e-15 … -1.1282853884771794e-14 -2.931553578721273e-15; -6.711912571546248e-25 -9.250882655161128e-14 … 2.3567754849818495e-15 9.252105575138463e-14], [-685462.4615372404 7.336565320060012e7 … 2.5391119879604124e7 7.336565320060046e7; 2.3539030391847983e7 -1.7607612212130383e8 … 1.3121185551060557e8 1.2201842685145742e8; … ; -4.869462514421187e-6 -4.850870778250243e-6 … -4.661907944682899e-

In [22]:
fig = Figure()
ax = Axis(fig[1,1]; title=@sprintf("Froude Number: %0.3f", Fr), xscale=log10)
norm_factor = grid.Lx * grid.Ly / (grid.nx^2 * grid.ny^2)

dealias!(Eh_flux, grid)
dealias!(E3h_flux, grid)
ρ1, EhrFF = FourierFlows.radialspectrum(Eh_flux, grid)
ρ2, E3hrFF = FourierFlows.radialspectrum(E3h_flux, grid)

dealias!(Ewwwh_flux, grid)
dealias!(E3wwwh_flux, grid)
ρ1, EwwwhrFF = FourierFlows.radialspectrum(Ewwwh_flux, grid)
ρ2, Ewww3hrFF = FourierFlows.radialspectrum(Ewww3h_flux, grid)

Πk = cumsum(EhrFF[end:-1:1])[end:-1:1]
Π3k = cumsum(E3hrFF[end:-1:1])[end:-1:1]
Πwwwk = cumsum(EwwwhrFF[end:-1:1])[end:-1:1]
Πwww3k = cumsum(Ewww3hrFF[end:-1:1])[end:-1:1]

lines!(ax, ρ1, norm_factor*Πk[:], color=:black, label=@sprintf("quadratic flux, Σ=%0.0e", FourierFlows.parsevalsum(Eh_flux, grid)))
lines!(ax, ρ2, norm_factor*Π3k[:], color=:red, label=@sprintf("cubic flux, Σ=%0.0e", FourierFlows.parsevalsum(E3h_flux, grid)))
lines!(ax, ρ1, norm_factor*Πwwwk[:], color=:black, linestyle=:dash)
lines!(ax, ρ2, norm_factor*Πwww3k[:], color=:red,  linestyle=:dash)


axislegend(ax, position=:rb)

save(@sprintf("images/flux_comparison_Fr=%0.3f.png", Fr), fig)
fig

LoadError: UndefVarError: `E3wwwh_flux` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [6]:
println(E3h_flux[1,1])
println(E3hrFF[1])

489667.8121917691


LoadError: UndefVarError: `E3hrFF` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [5]:
Nsnapshots = count_key_snapshots(run_directory, "rsw")

f0, Cg2 = read_rsw_params(run_directory)
ν, nν = read_rsw_dissipation(run_directory)
D = -ν*grid.Krsq.^(nν)
params = (; f=f0, Cg2, D)

t1, sol = load_key_snapshot(run_directory, "rsw", Nsnapshots-1)
sol = convert.(Complex{Float64}, sol)
uh = sol[:,:,1]
vh = sol[:,:,2]
ηh = sol[:,:,3]
rPE = rfft(sqrt.(1 .+ irfft(ηh, grid.nx)))
Eq1 = 0.5 * (abs2.(uh) + abs2.(vh) + params.Cg2*abs2.(rPE))
E1 = total_energy(uh, vh, ηh, grid, params)

t2, sol = load_key_snapshot(run_directory, "rsw", Nsnapshots)
sol = convert.(Complex{Float64}, sol)
uh = sol[:,:,1]
vh = sol[:,:,2]
ηh = sol[:,:,3]
rPE = rfft(sqrt.(1 .+ irfft(ηh, grid.nx)))
Eq2 = 0.5 * (abs2.(uh) + abs2.(vh) + params.Cg2*abs2.(rPE))
E2 = total_energy(uh, vh, ηh, grid, params)
println(sum(E2 - E1)/(t2 - t1))

-2.3044349120865963e7


In [262]:
println((FourierFlows.parsevalsum(E1, grid) - FourierFlows.parsevalsum(E2, grid))/(t2-t1))
#println((FourierFlows.parsevalsum(Eq1, grid) - FourierFlows.parsevalsum(Eq2, grid)))

-4.321241043715578e-6


In [43]:
println(FourierFlows.parsevalsum(E3h_flux, grid))
#println(sum(E3h_flux))
println(FourierFlows.parsevalsum(Eh_flux, grid))

3.2945152380084237e-12
0.0009925447765275813


In [257]:
heatmap(fftshift(log.(abs.()), 2))

LoadError: MethodError: no method matching abs()
The function `abs` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  abs([91m::Bool[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mbool.jl:153[24m[39m
[0m  abs([91m::Pkg.Resolve.VersionWeight[39m)
[0m[90m   @[39m [36mPkg[39m [90m/ext3/julia-1.11.5/share/julia/stdlib/v1.11/Pkg/src/Resolve/[39m[90m[4mversionweights.jl:32[24m[39m
[0m  abs([91m::Pkg.Resolve.FieldValue[39m)
[0m[90m   @[39m [36mPkg[39m [90m/ext3/julia-1.11.5/share/julia/stdlib/v1.11/Pkg/src/Resolve/[39m[90m[4mfieldvalues.jl:51[24m[39m
[0m  ...


In [188]:
dudth = params.f * vh - params.Cg2*1im*grid.kr .* ηh
dvdth = -params.f * uh - params.Cg2*1im*grid.l .* ηh
dηdth = -1im*grid.kr.*uh - 1im*grid.l.*vh
#dudt2 = params.f * vh - params.Cg2*1im*grid.kr.*ηh
dEhdt = 2*real.( conj.(uh) .* dudth + conj.(vh) .* dvdth + conj.(ηh) .* dηdth)
println(maximum(abs.(dEhdt)))

1.1920928955078125e-7


In [152]:
uh.*conj.(vh) + conj.(uh).*vh

513×1024 Matrix{ComplexF32}:
  1.37227f6+0.0im  -1.60121f8+0.0im  …  -1.02047f7+0.0im   1.81874f7+0.0im
  9.99531f7+0.0im  -1.04973f8+0.0im      7.40366f7+0.0im   1.96842f8+0.0im
  2.56511f6+0.0im  -6.70698f7+0.0im       1.4966f7+0.0im   8.25674f7+0.0im
 -1.12667f7+0.0im   1.17587f7+0.0im      6.97273f7+0.0im   1.27272f7+0.0im
  3.90741f7+0.0im  -2.48814f7+0.0im      3.13846f6+0.0im  -3.30504f6+0.0im
 -6.91074f6+0.0im  -2.65029f6+0.0im  …   2.88067f6+0.0im   1.59123f6+0.0im
 -1.86817f6+0.0im  -5.28497f6+0.0im      1.92416f7+0.0im    914689.0+0.0im
  7.75574f5+0.0im  -3.24586f6+0.0im     -3.85581f5+0.0im  -1.48376f5+0.0im
 -1.15245f6+0.0im    -3299.56+0.0im      2.80939f6+0.0im    -67468.5+0.0im
    84877.4+0.0im  -5.84878f5+0.0im      4.79423f5+0.0im     85017.0+0.0im
   214873.0+0.0im   -8.3453f5+0.0im  …   7.97978f5+0.0im  -2.75757f5+0.0im
  -1.7478f5+0.0im     62876.4+0.0im        50666.7+0.0im     38810.7+0.0im
    87387.4+0.0im    -3454.63+0.0im        62019.9+0.0im  -1.14642f5+0.

In [136]:
field = ones(grid.nkr, grid.nl)
dealias!(field, grid)
println(FourierFlows.parsevalsum(field, grid) / (grid.Lx * grid.Ly / (grid.nx^2 * grid.ny^2)))
r2, weights = create_radialspectrum_weights(grid, 1)
r, fieldr = FourierFlows.radialspectrum(field, grid, refinement=6) 
fieldr2 = radialspectrum(field, weights)
println(sum(fieldr2))
println((sum(fieldr))*(r[3]-r[2]))

115260.0
115260.0
115319.20580056983


In [129]:
r2

0.9999999721724656:0.9999999721724656:254.99999290397872

In [111]:
fieldr[1]

1.0

In [140]:
E3h_flux

257×512 Matrix{Float64}:
  43485.8             1.55117e7  …      -2.73835e6       1.55117e7
      1.14864e6      -3.23591e6          8.36288e6       1.08753e7
     -6.19234e6      -2.13222e7          4.59988e6       8.5097e6
      1.3304e6       -2.77078e6          3.93627e5       1.41635e6
      1.1157e6        5.55141e6         -2.23212e6       1.44311e7
     -8.28349e6       1.2879e6   …   -9970.7            -2.78106e6
      1.51099e6      -1.81666e5         -2.3474e6        2.08498e6
      1.73743e6      -7.54738e5          1.405e5         2.21258e5
      1.50173e5  246136.0               -6.01955e5      -5.63532e5
   8798.8            -1.70747e5          1.45009e5  -52137.7
     -1.4973e5   -10882.0        …  -78523.3        -95562.9
  20601.0            -1.30933e5     -27987.5        -26187.1
 -71786.7            -1.44499e5      24491.3         -3663.65
      ⋮                          ⋱       ⋮          
      0.0             0.0        …       0.0             0.0
      0.0     